In [7]:
import torch
from torch.autograd import Variable
from torch.autograd import backward

In [9]:
x = Variable(torch.randn(5,5))
y = Variable(torch.randn(5,5))
z = Variable(torch.randn(5,5), requires_grad=True)

In [10]:
a = x+y
a.requires_grad

False

In [11]:
b= a+z
b.requires_gradprint out.size()

True

In [12]:
from torchvision.models import resnet18

In [13]:
model = resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/manish/.torch/models/resnet18-5c106cde.pth
100.0%


In [14]:
for param in model.parameters():
    param.requires_grad = False

In [20]:
print type(model.fc)

<class 'torch.nn.modules.linear.Linear'>


In [21]:
from torch import optim,nn
model.fc = nn.Linear(512,100)
optimizer = optim.SGD(model.fc.parameters(), lr=1e-2, momentum=0.9)

In [22]:
regular_input = Variable(torch.randn(1, 3, 227, 227))
volatile_input = Variable(torch.randn(1, 3, 227, 227), volatile=True)
model(regular_input).requires_grad

True

In [23]:
model(volatile_input).requires_grad

False

In [24]:
b.grad_fn

In [26]:
x= torch.FloatTensor(5,7,3)
y= torch.FloatTensor(5,7,3)
print x.size()

torch.Size([5, 7, 3])


In [27]:
m = torch.add(torch.ones(4,1),torch.randn(4))
print m.size()

torch.Size([4, 4])


In [28]:
cx = torch.cuda.FloatTensor(1)
print cx


 0
[torch.cuda.FloatTensor of size 1 (GPU 0)]



In [29]:
with torch.cuda.device(0):
    ax = torch.cuda.FloatTensor(1)
    bx = torch.FloatTensor(1).cuda()
    cx = ax + bx

In [30]:
cx.get_device()

0

In [31]:
flag = torch.cuda.is_available()
print flag

True


In [32]:
x_gpu = torch.cuda.FloatTensor(1)
x_gpu.get_device()

0

In [36]:
y_gpu = x_gpu.new(x_gpu.size()).fill_(-2)
y_gpu.get_device()

0

In [65]:
from torch.autograd import Function
class LinearFunction(Function):
    @staticmethod
    def forward(ctx, input, weight, bias=None):
        ctx.save_for_backward(input, weight, bias)
        output = input.mm(weight.t())
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        input, weight, biprint out.size()as = ctx.saved_variables
        grad_input = grad_weight = grad_bias = None

        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0).squeeze(0)

        return grad_input, grad_weight, grad_bias

In [66]:
fn = LinearFunction.apply

In [67]:
from torch import nn
class Linear(nn.Module):
    def __init__(self,input_ft, output_ft,bias=True):
        super(Linear,self).__init__()
        self.input_ft = input_ft
        self.output_Ft = output_ft
        self.weight = nn.Parameter(torch.Tensor(output_ft,input_ft))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(output_ft))
        else:
            self.register_parameter('bias', None)
        self.weight.data.uniform_(-0.1, 0.1)
        if bias is not None:
            self.bias.data.uniform_(-0.1, 0.1)
    def forward(self, input):
        # See the autograd section for explanation of what happens here.
        return LinearFunction.apply(input, self.weight, self.bias)

In [74]:
model = Linear(20,10,True)
inp = Variable(torch.randn(1,20))
out = model(inp)


In [76]:
import numpy as np
a = np.arange(12)
b = torch.LongTensor(2,2)

In [77]:
print torch.is_tensor(a)
print torch.is_tensor(b)

False
True


In [78]:
torch.numel(b)

4

In [79]:
c= torch.from_numpy(a)
print torch.is_tensor(c)

True


In [81]:
x = torch.randn(2,3)
print x


-0.6781 -0.4659  0.9528
-0.6099 -0.9515 -1.4783
[torch.FloatTensor of size 2x3]



In [82]:
y = torch.randn(2,3)
print y


 1.3190  0.9288 -0.6080
 0.0137 -0.1713 -0.2642
[torch.FloatTensor of size 2x3]



In [83]:
z = torch.cat((x,y),0)
print z.size()

torch.Size([4, 3])


In [84]:
p = torch.cat((x,y),1)
print p.size()

torch.Size([2, 6])


In [85]:
x,y = torch.chunk(p,2,1)
print x.size()

torch.Size([2, 3])


In [86]:
t = torch.Tensor([[1,2],[3,4]])
torch.gather(t, 0, torch.LongTensor([[0,0],[1,0]]))


 1  2
 3  2
[torch.FloatTensor of size 2x2]

In [89]:
mat = torch.randn(3,4)
mask = mat.ge(0.5)
torch.masked_select(mat,mask)


 2.2999
 1.2576
 1.2601
[torch.FloatTensor of size 3]

In [91]:
xs = torch.zeros(2,1,3,1,2,1,2)
print "sz: ",xs.size()
ys = torch.squeeze(xs)
print "squ: ",ys.size()

sz:  torch.Size([2, 1, 3, 1, 2, 1, 2])
squ:  torch.Size([2, 3, 2, 2])


In [93]:
xp = torch.ones(2,2)
yp = torch.zeros(2,2)
sp = torch.stack((xp,yp))
print sp.size()

torch.Size([2, 2, 2])
